# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [25]:
# import required libraries
import numpy as np
import pandas as pd
import statistics as stats

Next, import `Orders` from Ironhack's database into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [26]:
# your code here

path = 'Orders.csv'

orders = pd.read_csv('Orders.csv')

orders.head()

Unnamed: 0  InvoiceNo StockCode  year  month  day  hour  \
0           0     536365    85123A  2010     12    3     8   
1           1     536365     71053  2010     12    3     8   
2           2     536365    84406B  2010     12    3     8   
3           3     536365    84029G  2010     12    3     8   
4           4     536365    84029E  2010     12    3     8   

                           Description  Quantity          InvoiceDate  \
0   white hanging heart t-light holder         6  2010-12-01 08:26:00   
1                  white metal lantern         6  2010-12-01 08:26:00   
2       cream cupid hearts coat hanger         8  2010-12-01 08:26:00   
3  knitted union flag hot water bottle         6  2010-12-01 08:26:00   
4       red woolly hottie white heart.         6  2010-12-01 08:26:00   

   UnitPrice  CustomerID         Country  amount_spent  
0       2.55       17850  United Kingdom         15.30  
1       3.39       17850  United Kingdom         20.34  
2       2.75       17850  United Kingdom         22.00  
3       3.39       17850  United Kingdom         20.34  
4       3.39       17850  United Kingdom         20.34

In [28]:
stats = orders.describe().T

stats['IQR'] = stats['75%'] - stats['25%']    #en este rango es donde esta la mitad de los datos

stats

count           mean            std       min        25%  \
Unnamed: 0    397924.0  278465.221859  152771.368303       0.0  148333.75   
InvoiceNo     397924.0  560617.126645   13106.167695  536365.0  549234.00   
year          397924.0    2010.934259       0.247829    2010.0    2011.00   
month         397924.0       7.612537       3.416527       1.0       5.00   
day           397924.0       3.614555       1.928274       1.0       2.00   
hour          397924.0      12.728247       2.273535       6.0      11.00   
Quantity      397924.0      13.021823     180.420210       1.0       2.00   
UnitPrice     397924.0       3.116174      22.096788       0.0       1.25   
CustomerID    397924.0   15294.315171    1713.169877   12346.0   13969.00   
amount_spent  397924.0      22.394749     309.055588       0.0       4.68   

                    50%        75%        max        IQR  
Unnamed: 0    284907.50  410079.25  541908.00  261745.50  
InvoiceNo     561893.00  572090.00  581587.00   22856.00  
year            2011.00    2011.00    2011.00       0.00  
month              8.00      11.00      12.00       6.00  
day                3.00       5.00       7.00       3.00  
hour              13.00      14.00      20.00       3.00  
Quantity           6.00      12.00   80995.00      10.00  
UnitPrice          1.95       3.75    8142.75       2.50  
CustomerID     15159.00   16795.00   18287.00    2826.00  
amount_spent      11.80      19.80  168469.60      15.12

---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [3]:
# your code here

Sub Problem 1: How to aggregate the amount_spent for unique customers?

In [10]:

customer_spending = orders.groupby('CustomerID')['amount_spent'].sum()
customer_spending


CustomerID
12346    77183.60
12347     4310.00
12348     1797.24
12349     1757.55
12350      334.40
           ...   
18280      180.60
18281       80.82
18282      178.05
18283     2094.88
18287     1837.28
Name: amount_spent, Length: 4339, dtype: float64

Sub Problem 2: How to select customers whose aggregated amount_spent is in a given quantile range?

In [23]:
#Calculate the aggregated amount spent for unique customers, as previously explained.

vip_quantile = customer_spending.quantile(0.95)  # Set the 95th percentile as the threshold for VIP.
preferred_menor = customer_spending.quantile(0.75)  # Set the 75th percentile as the lower threshold for Preferred.
preferred_mayor = customer_spending.quantile(0.95)  # Set the 95th percentile as the upper threshold for Preferred.

is_vip = customer_spending > vip_quantile
en_medio = (customer_spending >= preferred_menor) & (customer_spending <= preferred_mayor)


vip_customers = customer_spending[is_vip]
vip_customers


CustomerID
12346     77183.60
12357      6207.67
12359      6372.58
12409     11072.67
12415    124914.53
           ...    
18109      8052.97
18139      8438.34
18172      7561.68
18223      6484.54
18229      7276.90
Name: amount_spent, Length: 217, dtype: float64

In [24]:
preferred_customers = customer_spending[en_medio]

preferred_customers

CustomerID
12347    4310.00
12348    1797.24
12349    1757.55
12352    2506.04
12356    2811.43
          ...   
18259    2338.60
18260    2643.20
18272    3078.58
18283    2094.88
18287    1837.28
Name: amount_spent, Length: 868, dtype: float64

In [ ]:
#  Define the quantile range for VIP and Preferred Customers. 
#For example, you might choose the 95th percentile for VIP and the 75th to 95th percentiles for Preferred.

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [4]:
# your code here

In [ ]:
2. mayor cantidad de vips en cada cuartil

In [ ]:
is_vip = customer_data['Category'] == 'VIP'
is_vip = customer_spending > vip_quantile

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [5]:
# your code here

In [ ]:
# mejor combi de clienteS?

In [ ]:
#groupby